In [ ]:
"""

Script to format the data from the tesco extract into a useful excel (should have used csv!)

"""

In [ ]:
import pandas as pd
import json
import ast
import numpy as np
import os

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 25)

# Specify the folder containing the JSON files
input_folder_path = r"C:\Users\bened\OneDrive\Documents\Businesses\Relationship Predicting\Tesco Clubcards\3 - Raw Data Files"
output_folder_path = r"C:\Users\bened\OneDrive\Documents\Businesses\Relationship Predicting\Tesco Clubcards\4 - Processed Data Files"

In [8]:
filepath = r"C:\Users\bened\OneDrive\Documents\Businesses\Relationship Predicting\Tesco Clubcards\3 - Raw Data Files"

#Open JSON file
with open(f'{filepath}\Tesco-Customer-Data_Me.json') as f:
    data = json.load(f)

#Convert to DataFrame
df = pd.json_normalize(data)

<>:4: SyntaxWarning: invalid escape sequence '\T'
<>:4: SyntaxWarning: invalid escape sequence '\T'
C:\Users\bened\AppData\Local\Temp\ipykernel_29324\231643357.py:4: SyntaxWarning: invalid escape sequence '\T'
  with open(f'{filepath}\Tesco-Customer-Data_Me.json') as f:


In [10]:
#Extract first & second kev/val pair (customer info) - customer & spends
key1, dict1 = list(data.items())[0]
key2, dict2 = list(data.items())[1]

#Turn this into df and split out nested dicts
customer_df = pd.json_normalize(dict1)
spend_df = pd.json_normalize(dict2)

In [14]:
###TO DO
###Fix bug in which dicts within a list don't get split into new col

#Define the open_lists function
def open_lists(cell_value):
    if isinstance(cell_value, list):
        if cell_value and isinstance(cell_value[0], dict):
            return pd.json_normalize(cell_value[0])
        else:
            return ', '.join(map(str, cell_value))
    return cell_value

#Apply the function to each cell in the DataFrame
DIM_customer = customer_df.apply(lambda row: row.map(open_lists), axis=1)

In [15]:
#First turn basket data into df, dropping product and payment (categorised later)
#Split payment type list into its own df and assign timeStamp to later join on
#Create df for all info within product key
dfs = [(pd.DataFrame([data]).drop(columns=['product', 'paymentType']).assign(customer_email=customer_df.iloc[0, 0]),
        pd.json_normalize(data['paymentType'] if 'paymentType' in data else []).assign(timeStamp=data["timeStamp"]),
        pd.DataFrame(data["product"]).assign(timeStamp=data["timeStamp"], customer_email=customer_df.iloc[0, 0]))
       for data in dict2[0]]

#Separate DataFrames from the list
reference_dfs, payment_dfs, spend_dfs = zip(*dfs)

#Concatenate DataFrames along rows
final_reference_df = pd.concat(reference_dfs, ignore_index=True)
final_payment_df = pd.concat(payment_dfs, ignore_index=True)
final_spend_df = pd.concat(spend_dfs, ignore_index=True)

#Merge our reference and payment dfs
DIM_basket_df = pd.merge(final_reference_df, final_payment_df, on='timeStamp')

#Apply the open_lists function to handle lists within the DataFrames
DIM_basket_df = DIM_basket_df.apply(open_lists)
transaction_table = final_spend_df.apply(open_lists)
total_spend_df = pd.merge(transaction_table, DIM_basket_df, on=['timeStamp', 'customer_email'], how='left')


In [11]:
DIM_basket_df.to_excel("DIM_basket_Tesco.xlsx", index=False)
transaction_table.to_excel("Tesco_transactions.xlsx", index=False)
total_spend_df.to_excel("Tesco.xlsx", index=False)

In [ ]:
"""

Clean this up

"""